<a href="https://colab.research.google.com/github/waterbean0403/TIL/blob/main/05_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
"""# 트리의 앙상블"""

"""## 랜덤 포레스트"""

import numpy as np

import pandas as pd

from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [2]:
from sklearn.model_selection import cross_validate     # cross_validate() 함수를 사용해 교차 검증 수행

from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, random_state=42)     # RandomForestClassifier는 기본적으로 100개의 결정 트리를 사용하므로 n_jobs 매개변수를 -1로 지정하여 모든 CPU 코어를 사용

scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)     # cross_validate() 함수의 n_jobs 매개변수도 -1로 지정하여 최대한 병렬로 교차 검증을 수행. 또, return_train_score 매개변수를 True로 지정하여 훈련 세트에 대한 점수도 반환

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

# 훈련 세트에 다소 과대적합된 출력 결과

0.9973541965122431 0.8905151032797809


In [3]:
rf.fit(train_input, train_target)     # 앞의 랜덤 포레스트 모델을 훈련 세트에 훈련한 후 특성 중요도 출력
print(rf.feature_importances_)

# 05_1절의 특성 중요도와 비교했을 때, 두 번째 특성인 당도의 중요도가 감소하고 알코올 도수와 pH 특성의 중요도가 조금 상승
# 이런 이유는 랜덤 포레스트가 특성의 일부를 랜덤하게 선택하여 결정 트리를 훈련하기 때문
# 그 결과 하나의 특성에 과도하게 집중하지 않고 좀 더 많은 특성이 훈련에 기여할 기회를 얻음. 이는 과대적합을 줄이고 일반화 성능을 높이는 데 도움이 됨

[0.23167441 0.50039841 0.26792718]


In [4]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(train_input, train_target)
print(rf.oob_score_)

# 교차 검증에서 얻은 점수와 매우 비슷한 결과를 얻음. OOB 점수를 사용하면 교차 검증을 대신할 수 있어서 결과적으로 훈련 세트에 더 많은 샘플을 사용할 수 있음

0.8934000384837406


In [5]:
"""## 엑스트라 트리"""

from sklearn.ensemble import ExtraTreesClassifier     # 사이킷런에서 제공하는 엑스트라 트리

et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

# 랜덤 포레스트와 비슷한 결과를 얻음

0.9974503966084433 0.8887848893166506


In [6]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


In [7]:
"""## 그레이디언트 부스팅"""

from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

# 거의 과대적합이 되지 않음. 그레이디언트 부스팅은 결정 트리의 개수를 늘려도 과대적합에 매우 강함. 학습률을 증가시키고 트리의 개수를 늘리면 조금 더 성능이 향상될 수 있음

0.8881086892152563 0.8720430147331015


In [9]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

# 결정 트리 개수를 500개로 5개나 늘렸지만 과대적합을 잘 억제하고 있음

0.9464595437171814 0.8780082549788999


In [10]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

# 일반적으로 그레이디언트 부스팅이 랜덤 포레스트보다 조금 더 높은 성능을 얻을 수 있음. 하지만 순서대로 트리를 추가하기 때문에 훈련 속도가 느림

[0.15887763 0.6799705  0.16115187]


In [12]:
"""## 히스토그램 기반 그레이디언트 부스팅"""

from sklearn.experimental import enable_hist_gradient_boosting

from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

# 과대적합을 잘 억제하면서 그레이디언트 부스팅보다 조금 더 높은 성능 제공

0.9321723946453317 0.8801241948619236


In [13]:
from sklearn.inspection import permutation_importance     # 히스토그램 기반 그레이디언트 부스팅의 특성 중요도를 계산하기 위해 사용하는 함수

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)

# permutation_importance() 함수가 반환하는 객체는 반복하여 얻은 특성 중요도, 평균, 표준 편차를 담고 있음
# 평균을 출력해보면 랜덤 포레스트와 비슷한 비율임을 알 수 있음

[0.08876275 0.23438522 0.08027708]


In [14]:
result = permutation_importance(hgb, test_input, test_target, n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)

# 테스트 세트의 결과를 보면 그레이디언트 부스팅과 비슷하게 조금 더 당도에 집중하고 있음

[0.05969231 0.20238462 0.049     ]


In [15]:
hgb.score(test_input, test_target)

0.8723076923076923

In [17]:
"""### XGBoost"""

from xgboost import XGBClassifier

xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9558403027491312 0.8782000074035686


In [18]:
"""### LightGBM"""

from lightgbm import LGBMClassifier

lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.935828414851749 0.8801251203079884
